# Construct GAN and structure
---
Construct GAN, define training loop with multithreaded approach, and practice using industry standard terminal initialization commands

In [161]:
### - IMPORTS - ###
from PIL import Image
import numpy as np
import glob
from multiprocessing.dummy import Pool as TP
import cv2 as cv
import os
import random
### - other data augmentation imports - ### (if needed)
### - Imports - ###
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch as T
import torch.nn as nn
import torch.optim as optim #Neural network imports, multiply data etc
from torchvision.transforms import ToTensor
import torchvision.models as models
import torchvision
import torch.nn.functional as F #Neural Network used in Comp4660 at ANU

from Utils.NetworkHelpers import EqualizedLR_Conv2d, Pixel_norm, Minibatch_std

### - Other global variables - ###
LOVTV = [15, 26, 66] ##Training values to leave out

img_folder = '/Users/campb/Documents/PersonalProjects/AGRNet/Dataset/'

NS = '/Sample-'

image_format = 'RGB'

imsize = 4

multiplication_factor = 20

num_channels=3
kernal=4
s=2
p=1

In [171]:
# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
T.manual_seed(manualSeed)

Random Seed:  999


In [172]:
### - Train help functions - ###
#Cyclic LR Scheduler
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

factors = [1, 1, 1, 1, 1/2, 1/4, 1/8, 1/16, 1/32]

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
class fRGB(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.cvt = EqualizedLR_Conv2d(in_c, out_c, kernal_size=(1,1), stride=(1,1))
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        
    def forward(self, x):
        x = self.conv(x)
        return self.relu(x)
        
class tRGB(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.cvt = EqualizedLR_Conv2d(in_c, out_c, kernal_size=(1,1), stride=(1,1))
        
    def forward(self, x):
        return(self.cvt(x))

In [173]:
#Discriminator block
class D_Cell(nn.Module):
    def __init__(self, in_c, out_c, sb=0):
        self.sb = sb
        super().__init__()
        
        #Define network structure                                                         #initial block b (1-alpha)
        if sb == 0:
            #Set normal cell structure
            self.econv1 = EqualizedLR_Conv2d(in_c, out_c, kernel_size=(3,3), stride=(1,1), padding=(1,1)) #Initial block a (alpha)
            
            self.econv2 = EqualizedLR_Conv2d(out_c, out_c, kernel_size=(4,4), stride=(1,1))
            
            self.outlayer = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        else:
            self.mbstd = Minibatch_std()
            self.econv2 = EqualizedLR_Conv2d(out_c, out_c, kernel_size=(4,4), stride=(1,1)) #output block
            self.finish = nn.Sequential(nn.Flatten(), nn.Linear(out_c, out_c) ,nn.LeakyReLU(0.2, inplace=True), nn.Linear(out_c, 1))
        self.relu = nn.LeakyReLU(0.2, inplace=True)
    
    def forward(self, x):
        ### - Account for each discriminator block archetype - ###
        if self.sb == 0:
            x = self.econv1(x)
            x = self.relu(x)
            
            x = self.econv(x)
            x = self.relu(x)
            
            x = self.outlayer(x)
        else:
            x = self.mbstd(x)
            x = self.econv2(x)
            x = self.relu(x)
            x = self.finish(x)
        return x
            
#Generator Block

class G_Cell(nn.Module):
    def __init__(self, in_c, out_c, sb=0):
        self.sb = sb
        super().__init__()
        
        #Define network structure
        if sb == 0:
            self.us = nn.Upsample(scale_factor=2, mode='nearest') #Base block (standard cell)
            self.conv1 = EqualizedLR_Conv2d(in_c, out_c, kernel_size=(3,3), stride=(1,1), padding='same')
            self.conv2 = EqualizedLR_Conv2d(in_c, out_c, kernel_size=(3,3), stride=(1,1), padding='same')
        elif sb == 1:
            self.dense = nn.Linear(in_c) #Our first initial training layer
            self.conv1 = EqualizedLR_Conv2d(in_c, out_c, kernel_size=(3,3), stride=(1,1), padding='same')
            
        
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.pn = Pixel_norm()
        
    def forward(self, x):
        if self.sb == 0:
            x = self.us(x)
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pn(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pn(x)
        elif self.sb == 1:
            x = self.pn(x)
            x = self.dense(x)
            x = self.relu(x)
            x = self.pn(x)
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pn(x)
        return x
            

# Network Structure and Basic Theory

---

Each network will progressively need to grow more and more in order to upscale the images while keeping each of the dims the same for upscaling it.
Therefore to accurately train this model we construct it in such a way that we may output a 1 megapixel image.

This involves defining the structure for the overall network once it is finished and including a depth index variable that will be increased in training.

In [174]:
#Discriminator and Generator#
class G(nn.Module):
    def __init__(self, ls, out):
        """
        ls is latent size
        out is desired output resolution
        build structure iteratively
        """
        super().__init()
        self.depth = 1 #Current indexing
        self.alpha = 1 #Fade value
        self.incalpha = 0 #Value to increment alpha by
        
        self.trgb = tRGB(ls, 3) #torgb value
        self.us = nn.Upsample(scale_factor=2, mode='nearest')
        self.net = nn.ModuleList([G_Cell(ls, ls, sb=1)])
        self.rgbs = nn.ModuleList([tRGB(latent_size, 3)])
        
        #Add all standard blocks
        for i in range(2, int(np.log2(out))):
            ### - trick is to decrease the latent vector as well for each of the higher level blocks - ###
            if i < 7: 
                in_c = 1024
                out_c = 1024
            else:
                in_c = int(1024 / 2**(i-7))
                out_c = int(1024 / 2**(i-7))
            self.net.append(G_Cell(in_c, out_c))
            self.rgbs.append(tRGB(out_c, 3))
            
    def forward(self, x):
        for cell in self.net[:self.depth-1]:
            x = cell(x)
        out = self.net[self.depth-1](x)
        crgb = self.rgbs[self.depth-1](out)
        if self.alpha < 1:
            xprev = self.us(x)
            rgbprev = self.rgbs[self.depth-2](xprev)
            crgb = self.alpha * (rgbprev) + (1-self.alpha)*(crgb)
        return crgb
    def inc_depth(self, iters):
        self.incalpha = 1/iters
        self.alpha = 1/iters
        self.depth += 1

class D(nn.Module):
    def __init__(self,ls, out):
        super().__init__()
        self.depth = 1
        self.alpha = 1
        self.incalpha = 0
        
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.ds = nn.AvgPool2d(kernel_size(2,2), stride(2,2))
        
        self.net = nn.ModuleList([D_Cell(ls, ls, sb=3)]) #initialize final block
        self.frgbs = nn.ModuleList([fRGB(3, ls)])
        
        for i in range(2, int(np.log2(out))):
            if i < 7:
                in_c, out_c = 1024, 1024
            else:
                in_c, out_c = int(512 / 2**(i - 5)), int(512 / 2**(i - 6))
                
            self.net.append(D_Cell(in_c, out_c))
            self.frgbs.append(fRGB(3, in_c))
            
        def forward(self, x):
            xc = self.frgbs[self.depth-1](x)
            xc = self.net[self.depth-1](xc)
            if self.alpha < 1: #if depth != 1
                x = self.ds(x)
                xprev = self.frgbs[self.depth-2](x)
                xprev = self.relu(xprev)
                xc = self.alpha*xprev + self.alpha*xc
            for cell in reversed(self.net[:self.depth-1]):
                xc = cell(xc)
            
            return xc
        
        def inc_depth(self, iters):
            self.incalpha = 1/iters
            self.alpha = 1/iters
            self.depth += 1
        